Kvasir dataset split into neg/pos and trained using Resnet50 without augmentation. Getting some decent results after training on resampled data with large step-size.  
- Class weighting  
- Resampling  
- Initial Bias-estimation
- Decreasing learning rate

### Loading data

In [22]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt

# Some stuff to make utils-function work
import sys
sys.path.append('../utils')
from data_prep import create_dataset, print_class_info, show_image
%load_ext autoreload
%autoreload 2

# Jupyter-specific
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
data_dir = pathlib.Path('/mnt/sdb/hyper-kvasir/labeled/')

image_count = len(list(data_dir.glob('*/*.*g')))
print (image_count)

BATCH_SIZE = 1024
IMG_HEIGHT = 64
IMG_WIDTH = 64
num_classes = 23

10662


In [41]:
class_names = np.array([item.name for item in data_dir.glob('*') if item.name != '*.txt'])

# Create a dataset of the file paths
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

A short pure-tensorflow function that converts a file path to an `image_data, label` pair:

In [49]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # get class integer from class-list
    label_int = tf.reduce_min(tf.where(tf.equal(parts[-2], class_names)))
    # cast to tensor array with dtype=uint8
    return tf.dtypes.cast(label_int, tf.int32)

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

# Set 'num_parallel_calls' so multiple images are loaded and processed in parallel
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

### Prepare dataset for training
Want the data to be shuffled and batched. Here we use the `tf.data` api.

In [50]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
      if isinstance(cache, str):
        ds = ds.cache(cache)
      else:
        ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

train_ds = prepare_for_training(labeled_ds, cache="./cache/resampler_64.tfcache")

### Resampling

In [70]:
def count(counts, batch):
    images, labels = batch
    
    for i in range(num_classes):
        counts['class_{}'.format(i)] += tf.reduce_sum(tf.cast(labels == i, tf.int32))

    return counts

In [71]:
# Set the initial states
initial_state = {}
for i in range(num_classes):
    initial_state['class_{}'.format(i)] = 0

# Count samples
counts = train_ds.take(10).reduce(
    initial_state = initial_state,
    reduce_func = count)


final_counts = []
for class_, value in counts.items():
    final_counts.append(value.numpy().astype(np.float32))

final_counts = np.asarray(final_counts)

fractions = final_counts/final_counts.sum()
print(fractions)

[0.00058594 0.00400391 0.03837891 0.02441406 0.00341797 0.00498047
 0.09404297 0.09345703 0.03720703 0.04072266 0.01865234 0.10751953
 0.06074219 0.00087891 0.07216797 0.08730469 0.00263672 0.01191406
 0.09580078 0.09355469 0.09423828 0.01240234 0.00097656]


### Resampling - dataset as two different tf.data datasets

In [96]:
datasets = []
for i in range(num_classes):
    ds = train_ds.unbatch().filter(lambda image, label: label==i).repeat()
    datasets.append(ds)

In [100]:
for image, label in datasets[2].batch(10).take(1):
    print (label.numpy())

[2 2 2 2 2 2 2 2 2 2]


In [101]:
balanced_ds = tf.data.experimental.sample_from_datasets(datasets, target_dist).batch(10)

In [102]:
for images, labels in balanced_ds.take(10):
    print (labels.numpy())

[ 7 21  3  3  4 17  2  7 19  6]
[ 1 13 13 14 17  0 21 20 14 22]
[13  6  7 14  7 17  8  1 12  2]
[15  3 14 20  1 14 17  5  6 12]
[22  4 17  9 20  9  7  7 20 22]
[ 5 22  6  0 11 16 14  7  9  4]
[12 15  5 18  3 12 16  2  7 14]
[ 6  7 11  2  4 21 14 18 12  8]
[21 11 20  7  5 21  6 10  4  1]
[ 9  8  1 14  9  8  6  9  9 12]


In [103]:
initial_state = {}
for i in range(num_classes):
    initial_state['class_{}'.format(i)] = 0

counts = balanced_ds.take(30).reduce(
    initial_state = initial_state,
    reduce_func = count)


final_counts = []
for class_, value in counts.items():
    final_counts.append(value.numpy().astype(np.float32))

final_counts = np.asarray(final_counts)

fractions = final_counts/final_counts.sum()
print (fractions)

[0.03666667 0.04       0.07666667 0.06333333 0.06       0.03333334
 0.04666667 0.05333333 0.04       0.04333333 0.02333333 0.03666667
 0.04333333 0.03333334 0.06       0.04666667 0.03666667 0.03
 0.03       0.03       0.05333333 0.04333333 0.04      ]


### Rejection resampling

In [88]:
def class_func(image, label):
    return tf.cast(label, tf.int32)

In [89]:
target_dist = []
for i in range(num_classes):
    target_dist.append(1.0/num_classes)

resampler = tf.data.experimental.rejection_resample(
            class_func, #=lambda features, label: label, 
            target_dist=target_dist,
            initial_dist=fractions)

In [90]:
resample_ds = train_ds.unbatch().apply(resampler).batch(10)

In [91]:
# Testing cell
for labels_resampled, img_and_label in resample_ds.take(1):
    print(img_and_label[1])

tf.Tensor([14 11 15 12 15 18 11 15  8  7], shape=(10,), dtype=int32)


In [92]:
labels_resampled

<tf.Tensor: id=4098, shape=(10,), dtype=int32, numpy=array([ 8,  0, 14, 18, 21, 15,  8, 16,  6,  1], dtype=int32)>

In [93]:
img_and_label[1]

<tf.Tensor: id=4100, shape=(10,), dtype=int32, numpy=array([14, 11, 15, 12, 15, 18, 11, 15,  8,  7], dtype=int32)>

In [94]:
balanced_ds = resample_ds.map(lambda extra_label, img_and_label: img_and_label)

In [95]:
for images, labels in balanced_ds.take(10):
    print(labels.numpy())

[18  6 12 20  9 12 11 15  3 15]
[18 20 20  9  1  7 12  7 12 14]
[15 18 20  9  8 11 19  7 11 15]
[ 7 19  6  6 18 20 15  6 18  6]
[19 18 15 20  3  6 11 19 18 15]
[19 20 12 21 21 12 11  6 14 11]
[16 19  8 15 15 11 15  3  3  7]
[14 19 12 20 14 18 20  6 20 18]
[19  6 15 11 11  7  6  9 15 11]
[14 15 12 11  2 14  9 15  3  1]


In [66]:
def count(counts, batch):
    images, labels = batch
    
    for i in range(num_classes):
        counts['class_{}'.format(i)] += tf.reduce_sum(tf.cast(labels == i, tf.int32))

    return counts

In [69]:
initial_state = {}
for i in range(num_classes):
    initial_state['class_{}'.format(i)] = 0

counts = balanced_ds.take(30).reduce(
    initial_state = initial_state,
    reduce_func = count)


final_counts = []
for class_, value in counts.items():
    final_counts.append(value.numpy().astype(np.float32))

final_counts = np.asarray(final_counts)

fractions = final_counts/final_counts.sum()
print(fractions)

[0.04666667 0.04666667 0.03666667 0.04333333 0.06333333 0.05
 0.04       0.04       0.05       0.03666667 0.04       0.04666667
 0.05       0.05       0.03333334 0.02333333 0.04333333 0.04
 0.04       0.03333334 0.05666667 0.05666667 0.03333334]
